In [1]:
!gdown 1WGaxwrpoBzvS3njzm-rBp80cCqwapyZw # веса модели

Downloading...
From: https://drive.google.com/uc?id=1WGaxwrpoBzvS3njzm-rBp80cCqwapyZw
To: /content/ViT_2.pt
100% 343M/343M [00:08<00:00, 38.3MB/s]


In [2]:
!lscpu | grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz


In [ ]:
!pip install transformers datasets

In [ ]:
!python -m pip install git+https://github.com/huggingface/optimum-intel.git

In [5]:
import os

import torch
from torch import nn
import torch.nn.utils.prune as prune

from torch.utils.data import Dataset, DataLoader

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import ViTImageProcessor, ViTForImageClassification
from datasets import load_dataset

from tqdm.notebook import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Подготовка

In [34]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = nn.Linear(768, 2)

model.config.id2label = {0: 0, 1: 1}

In [35]:
model.load_state_dict(torch.load('ViT_2.pt', map_location=torch.device('cpu')))
model.eval();

In [61]:
model_dir = 'model_dir'

model.save_pretrained(model_dir)

In [16]:
ds = load_dataset('cats_vs_dogs')

In [27]:
indexes = list(range((len(ds['train']))))
train, test = train_test_split(indexes, test_size=0.05, random_state=0)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, ids, dataset):
        self.ids = ids
        self.ds = dataset

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
      image = self.ds['train'][index]['image']
      label = self.ds['train'][index]['labels']

      image = processor(
          image.convert("RGB"),
          return_tensors='pt'
          )

      image['pixel_values'] = image['pixel_values'].squeeze(0)

      return image, label

In [ ]:
val_dataset = CustomDataset(
    ids=test,
    dataset=ds
)

In [ ]:
val_loader = DataLoader(val_dataset, batch_size=50, shuffle=False, num_workers=2)

In [ ]:
def get_model_size(model) -> None:
    model_filepath = 'tmp.pth'
    torch.save(model.state_dict(), model_filepath)
    size = os.path.getsize(model_filepath) / 1024 ** 2
    print(f'Model size (MB): {size:.5f}')
    os.remove(model_filepath)

# До применения методов:

In [ ]:
val_targets = []
val_preds = []

with torch.no_grad():
    for i, (batch, targets) in enumerate(tqdm(val_loader)):

        batch = batch.to('cpu')
        targets = targets.to('cpu')

        outputs = model(**batch)
        logits = outputs.logits
        val_targets.extend(targets.cpu().numpy())
        val_preds.extend(logits.argmax(axis=1).cpu().numpy())

print('F1:', f1_score(val_targets, val_preds, average='macro'))

  0%|          | 0/24 [00:00<?, ?it/s]

F1: 1.0


In [ ]:
sample = next(iter(val_loader))[0]['pixel_values'][:1]

In [ ]:
%%timeit

with torch.no_grad():
    _ = model(sample)

1.09 s ± 17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
get_model_size(model)

Model size (MB): 327.36333


# Transformers Pipeline:

In [79]:
from transformers import pipeline

In [80]:
pipe = pipeline(
    task='image-classification',
    model=model,
    image_processor=processor,
    framework='pt',
    device='cpu'
)

In [ ]:
pbar = tqdm(total=len(test))

val_targets = []
val_preds = []

with torch.no_grad():
    for index in test:
        img = ds['train'][index]['image']
        target = ds['train'][index]['labels']
        output = pipe(images=img)[0]['label']

        val_targets.append(target)
        val_preds.append(output)

        pbar.update(1)

print('F1:', f1_score(val_targets, val_preds, average='macro'))

  0%|          | 0/1171 [00:00<?, ?it/s]

F1: 0.9991457275067719


In [ ]:
sample = ds['train'][0]['image']

In [ ]:
%%timeit

with torch.no_grad():
    _ = pipe(images=sample)

783 ms ± 6.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
def get_model_size(model) -> None:
    model_filepath = 'tmp'
    model.save_pretrained(model_filepath)

In [85]:
get_model_size(pipe)

size = os.path.getsize('tmp/pytorch_model.bin') / 1024 ** 2
print(f'Model size (MB): {size:.5f}')

!rm -rf tmp

Model size (MB): 327.36436


# Optimum Pipeline

In [59]:
from optimum.pipelines import pipeline

In [ ]:
optimum_pipe = pipeline(
    task='image-classification',
    model=model_dir,
    image_processor='google/vit-base-patch16-224',
    framework='pt',
    device='cpu',
    accelerator="ort",
)

In [40]:
pbar = tqdm(total=len(test))

val_targets = []
val_preds = []

with torch.no_grad():
    for index in test:
        img = ds['train'][index]['image']
        target = ds['train'][index]['labels']
        output = optimum_pipe(images=img)[0]['label']

        val_targets.append(target)
        val_preds.append(output)

        pbar.update(1)

print('F1:', f1_score(val_targets, val_preds, average='macro'))

  0%|          | 0/1171 [00:00<?, ?it/s]

F1: 0.9991457275067719


In [41]:
sample = ds['train'][0]['image']

In [42]:
%%timeit

with torch.no_grad():
    _ = optimum_pipe(images=sample)

499 ms ± 64.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [88]:
get_model_size(optimum_pipe)

size = os.path.getsize('tmp/model.onnx') / 1024 ** 2
print(f'Model size (MB): {size:.5f}')

!rm -rf tmp

Model size (MB): 327.55270


# ORTModelForImageClassification

In [66]:
import onnxruntime

from optimum.onnxruntime import ORTModelForImageClassification, ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

session_options = onnxruntime.SessionOptions()
session_options.log_severity_level = 0

In [ ]:
optimum_pipe_1 = ORTModelForImageClassification.from_pretrained(
    model_id=model_dir,
    export=True,
    provider="CPUExecutionProvider",
    session_options=session_options
)

In [67]:
optimization_config = OptimizationConfig(
    optimization_level=99
)

optimizer = ORTOptimizer.from_pretrained(optimum_pipe_1)
optimizer.optimize(save_dir='optimized_model', optimization_config=optimization_config)
optimized_model = ORTModelForImageClassification.from_pretrained('optimized_model')

In [54]:
onnx_pipe = pipeline(
    "image-classification",
    model=optimized_model,
    image_processor='google/vit-base-patch16-224',
    feature_extractor='google/vit-base-patch16-224',
    accelerator="ort",
    device='cpu')

In [55]:
pbar = tqdm(total=len(test))

val_targets = []
val_preds = []

with torch.no_grad():
    for index in test:
        img = ds['train'][index]['image']
        target = ds['train'][index]['labels']
        output = onnx_pipe(images=img)[0]['label']

        val_targets.append(target)
        val_preds.append(output)

        pbar.update(1)

print('F1:', f1_score(val_targets, val_preds, average='macro'))

  0%|          | 0/1171 [00:00<?, ?it/s]

F1: 0.9991457275067719


In [56]:
sample = ds['train'][0]['image']

In [57]:
%%timeit

with torch.no_grad():
    _ = onnx_pipe(images=sample)

362 ms ± 9.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
size = os.path.getsize('optimized_model/model_optimized.onnx') / 1024 ** 2
print(f'Model size (MB): {size:.5f}')

Model size (MB): 327.34057
